# Neural Machine Translation — Experiments
Workbook for experimenting with small scale systems.

In [ ]:
%%capture
!pip install watermark
!pip install evaluate
!pip install sacrebleu
!pip install sacremoses
!pip install wandb
!pip install peft
!pip install transformers sentencepiece
!pip install -U transformers
# !pip install --force-reinstall --upgrade --no-cache-dir --no-deps unsloth unsloth_zoo

## 0. Configuration

In [ ]:
def is_colab():
  try:
    import google.colab

    return True
  except ImportError as e:
    print(f"{e}", end='')
    return False

google_colab = is_colab()

if google_colab:
    from google.colab import drive, userdata
    from huggingface_hub import login
    import wandb

    drive.mount('/content/gdrive')
    HF_TOKEN = userdata.get('HF_TOKEN')
    WAND_TOKEN = userdata.get('wandb')

    login(token=HF_TOKEN)
    wandb.login(key=WAND_TOKEN)

Mounted at /content/gdrive


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wzmatt (wzmatt-university-of-arizona) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
%reload_ext watermark

import random
from dataclasses import dataclass
from time import perf_counter
from tkinter import filedialog
import pdb
import os
import shutil

import pandas as pd
import evaluate
import torch
import numpy as np
from datasets import load_dataset, get_dataset_config_names, Dataset, DatasetDict, concatenate_datasets
import accelerate
from transformers.trainer_utils import get_last_checkpoint
from peft import PeftModelForSeq2SeqLM, get_peft_config
from transformers import (
    MarianMTModel, MarianTokenizer, MarianConfig,
    Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq,
    set_seed,
)

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

%watermark -a "Matthew" -d -u -v -p pandas,evaluate,torch,numpy,datasets,transformers,wandb,accelerate,huggingface_hub

Author: Matthew

Last updated: 2025-03-31

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

pandas         : 2.2.2
evaluate       : 0.4.3
torch          : 2.6.0+cu124
numpy          : 2.0.2
datasets       : 3.5.0
transformers   : 4.50.3
wandb          : 0.19.8
accelerate     : 1.5.2
huggingface_hub: 0.29.3



In [ ]:
@dataclass
class Config:
  """Configuration for machine translation experiments. Intended to be reconfigurated for each run."""
  seed: int = 42

  source_lang: str = "en"
  target_lang: str = "es"
  cache_dir: str = "./gdrive/MyDrive/translation/"

  dataset: str = "weezygeezer/Spanish-Asturian_Parallel-Corpus"
  task_prefix: str = ">>spa<< "
  task_prefix_2: str = ">>ast<< "
  label_pad_token_id: int = ""
  max_source_length: int = 512
  max_target_length: int = 512
  model_max_length: int = 0

  batch_size: int = 1
  lr: float = 1e-4
  num_epochs: int  = 5
  weight_decay: float = 0.01
  num_beams: int = 4
  save_steps: int = 20_000

  model_checkpoint = 'Helsinki-NLP/opus-mt-tc-bible-big-deu_eng_fra_por_spa-itc'
  device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  do_train: bool = True

  def __post_init__(self):
      """Sets the random seed upon class creation."""
      random.seed(self.seed)
      np.random.seed(42)
      torch.manual_seed(self.seed)
      torch.cuda.manual_seed_all(self.seed)
      set_seed(self.seed)

      print(f"The device {self.device} is being used.", end="\n")

config = Config()

The device cuda is being used.


In [ ]:
model_name = config.model_checkpoint.split("/")[-1]
output_dir = os.path.join(config.cache_dir, f"{model_name}_{config.source_lang}-{config.target_lang}")

fine_tuned_model_checkpoint = os.path.join(
    output_dir,
    "checkpoint")

def make_dir():
  try:
    os.makedirs(fine_tuned_model_checkpoint, exist_ok=False)
    print(f"Directory {fine_tuned_model_checkpoint} created.")
  except OSError as error:
    shutil.rmtree(fine_tuned_model_checkpoint)
    os.makedirs(fine_tuned_model_checkpoint, exist_ok=False)
    print(f"The previous {fine_tuned_model_checkpoint} directory was removed to create new directory.")

# make_dir()

In [ ]:
chrf = evaluate.load('chrf', cache_dir=config.cache_dir)
bleu_score = evaluate.load('bleu', cache_dir=config.cache_dir)
sacrebleu_score = evaluate.load('sacrebleu', cache_dir=config.cache_dir)

In [ ]:
def generate_translation(batch):
  """
  Translation function for evaluation.
  """
  inputs = [x for x in batch[config.source_lang]]
  inputs = [config.task_prefix_2 + x for x in inputs]

  encoded = tokenizer(
      inputs,
      max_length=tokenizer.model_max_length,
      truncation=True,
      padding=True,
      return_tensors='pt',
  )

  input_ids = encoded.input_ids.to(device)
  attention_mask = encoded.attention_mask.to(device)

  output = translator.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      num_beams=4,
      max_length=tokenizer.model_max_length,
  )

  decoded = tokenizer.batch_decode(
      output,
      skip_special_tokens=True
  )

  targets = [x for x in batch[config.target_lang]]

  return {
      'reference': targets,
      'prediction': decoded
  }

In [ ]:
def generate_backtranslation(batch):
  """
  Function for backtranslation.
  **Notably, the reference is the source language.**
  """
  inputs = [x for x in batch[config.source_lang]]
  references = inputs.copy()
  inputs = [task_prefix_2 + x for x in inputs]

  encoded = tokenizer(
      inputs,
      max_length=tokenizer.model_max_length,
      truncation=True,
      padding=True,
      return_tensors='pt',
  )

  input_ids = encoded.input_ids.to(device)
  attention_mask = encoded.attention_mask.to(device)

  output = translator.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      num_beams=4,
      max_length=tokenizer.model_max_length,
  )

  decoded = tokenizer.batch_decode(
      output,
      skip_special_tokens=True,
  )

  return {
      "reference": references,
      "prediction": decoded,
  }

In [ ]:
def backtranslate(batch):
  inputs = [x for x in batch['asturian']]
  inputs = [task_prefix + x for x in inputs]

  encoded = tokenizer(
      inputs,
      max_length=tokenizer.model_max_length,
      truncation=True,
      padding=True,
      return_tensors='pt',
      )

  input_ids = encoded.input_ids.to(device)
  attention_mask = encoded.attention_mask.to(device)

  output = translator.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      num_beams=1,
      max_length=tokenizer.model_max_length,
      )

  decoded = tokenizer.batch_decode(
      output,
      skip_special_tokens=True
      )

  targets = [x for x in batch['asturian']]

  return {
      source: decoded,
      target: targets,
        }

## 0. Backtranslation

In [ ]:
backtranslation = False
source = 'Spanish'
target = 'Asturian'

In [ ]:
if backtranslation:
  device = torch.device('cuda')
  task_prefix = ">>spa<< "
  # model_name = 'Helsinki-NLP/opus-mt-tc-bible-big-roa-deu_eng_fra_por_spa'
  model_name = 'Helsinki-NLP/opus-mt-tc-bible-big-itc-fra_ita_por_spa'
  translator = MarianMTModel.from_pretrained(model_name)
  translator.to(device)
  tokenizer = MarianTokenizer.from_pretrained(model_name)


In [ ]:
if backtranslation:
  monolingual_corpus = 'literary_cln.ast'
  dataset = load_dataset(path='csv', data_files=monolingual_corpus, encoding='utf-8',
                       delimiter='\t', column_names=['asturian', 'idx'],
                       split='train')
  dataset = dataset.remove_columns('idx')


In [ ]:
def backtranslate(batch):
  inputs = [x for x in batch['asturian']]
  inputs = [task_prefix + x for x in inputs]

  encoded = tokenizer(
      inputs,
      max_length=tokenizer.model_max_length,
      truncation=True,
      padding=True,
      return_tensors='pt',
      )

  input_ids = encoded.input_ids.to(device)
  attention_mask = encoded.attention_mask.to(device)

  output = translator.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      num_beams=1,
      max_length=tokenizer.model_max_length,
      )

  decoded = tokenizer.batch_decode(
      output,
      skip_special_tokens=True
      )

  targets = [x for x in batch['asturian']]

  return {
      source: decoded,
      target: targets,
        }

In [ ]:
if backtranslation:
  results = dataset.map(
      backtranslate,
      batched=True,
      batch_size=125,
      remove_columns=dataset.column_names
  )

In [ ]:
if backtranslation:
  test = results.to_pandas()
  test.to_csv('literary_bitext.csv', header=True, index=False, encoding='utf-8-sig')

## 0. Load train & evaluation (Flores)
- datasets should already be filtered during this process
- depending on target language, can use flores, opus, or a held-out dataset for evaluation

In [ ]:
def extract_langs(row):
  return {
      "en": row['translation'].get("en", ""),
      "es": row['translation'].get("es", "")
  }

def create_flores(flores, pair):
  evaluation = {}

  for eval in flores:
    dev_src = flores[eval].filter(lambda x: x['iso_639_3'].startswith(pair['src'][0]))
    dev_tgt = flores[eval].filter(lambda x: x['iso_639_3'].startswith(pair['tgt'][0]))
    assert len(dev_src) == len(dev_tgt)

    eval_ds = Dataset.from_dict(
        {pair['src'][1]: dev_src['text'],
         pair['tgt'][1]: dev_tgt['text']}
    )
    evaluation[eval] = eval_ds

  return evaluation

In [ ]:
dev_ast = load_dataset('text', data_files='gdrive/MyDrive/translation/FLORES+/dev/dev.ast_Latn', split='train')
dev_spa = load_dataset('text', data_files='gdrive/MyDrive/translation/FLORES+/dev/dev.spa_Latn', split='train')

In [ ]:
dev_ast = dev_ast.rename_column('text', 'ast')
dev_spa = dev_spa.rename_column('text', 'es')

In [ ]:
dev_ds = concatenate_datasets([dev_spa, dev_ast], axis=1)

In [ ]:
# %%capture
# flores = load_dataset('openlanguagedata/flores_plus')

# pair = {'src': ('spa', 'es'), 'tgt': ('ast', 'ast')}
# flores_eval = create_flores(flores, pair)

In [ ]:
# dev_ds = flores_eval['dev']
# devtest_ds = flores_eval['devtest']

In [ ]:
train_ds = load_dataset(config.dataset, split="GNOME+KDE4+OpenSubtitles+PILARcrawled+PILARliterary+Tatoeba+wikimedia")

In [ ]:
train_ds = train_ds.rename_column('Spanish', 'es')
train_ds = train_ds.rename_column('Asturian', 'ast')

In [ ]:
dataset_dict = DatasetDict({'train': train_ds,
             'dev': dev_ds,
            #  'devtest': devtest_ds
                            })

In [ ]:
# dataset_dict['dev'] = dev_ds
# dataset_dict['devtest'] = devtest_ds

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['es', 'ast'],
        num_rows: 122449
    })
    dev: Dataset({
        features: ['es', 'ast'],
        num_rows: 997
    })
})

## 1. Baseline System
- Apertium
- Baseline (zero-shot translation/cross-lingual transfer)
  - Marian MT: roa-en, big-mul-mul, and big-roa-few
  - Performance: https://opus.nlpl.eu/dashboard/index.php?pkg=opusmt&test=all&scoreslang=all&chart=standard&model=Tatoeba-MT-models/mul-mul/opusTCv20230926%2Bbt%2Bjhubc_transformer-big_2024-08-17
  - https://huggingface.co/Helsinki-NLP/opus-mt-tc-bible-big-mul-mul
  - https://huggingface.co/docs/transformers/en/model_doc/marian

In [ ]:
baseline = True

if baseline:
  baseline_config = Config()

The device cuda is being used.


In [ ]:
if baseline:
  baseline_config.model_checkpoint = 'Helsinki-NLP/opus-mt-tc-bible-big-deu_eng_fra_por_spa-itc'
  baseline_config.source_lang = 'es'
  baseline_config.target_lang = 'ast'

In [ ]:
%%capture
if baseline:
  tokenizer = MarianTokenizer.from_pretrained(baseline_config.model_checkpoint)
  model = MarianMTModel.from_pretrained(baseline_config.model_checkpoint)
  model.to(baseline_config.device)

  baseline_config.model_max_length = tokenizer.model_max_length
  baseline_config.task_prefix = ">>spa<< " #may need to test again because the space wasn't included ?
  baseline_config.task_prefix_2 = ">>ast<< "
  baseline_config.num_beams = 4
  # baseline_config.batch_size = 1 #not needed

In [ ]:
def translate(batch): # might need to update later on for trained model
  inputs = [baseline_config.task_prefix + x for x in batch['es']]

  encoded = tokenizer(
      inputs,
      max_length=baseline_config.model_max_length,
      truncation=True,
      padding=True,
      return_tensors="pt"
  )
  input_ids = encoded.input_ids.to(config.device)
  attention = encoded.attention_mask.to(config.device)

  output = model.generate(
      input_ids = input_ids,
      attention_mask = attention,
      num_beams=baseline_config.num_beams,
      max_length=baseline_config.model_max_length,
  )

  decoded = tokenizer.batch_decode(
      sequences=output,
      skip_special_tokens=True
  )

  targets = [[x] for x in batch['ast']]
  return {
      "reference": targets,
      "prediction": decoded
  }

In [ ]:
# test_ds

In [ ]:
if baseline:
  results = dev_ds.map(
    translate,
    batched=True,
    batch_size=30,
    remove_columns=dev_ds.column_names)

  metric = evaluate.load("sacrebleu")
  chrf = evaluate.load("chrf")

  for r in results:
    prediction = r['prediction']
    reference = r['reference']
    metric.add(prediction=prediction, reference=reference)
    chrf.add(prediction=prediction, reference=reference)

  bleu = metric.compute()
  chr2 = chrf.compute(word_order=2)

  results = {'bleu': bleu['score']}
  results['chrf2'] = chr2['score']

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

In [ ]:
results

{'bleu': 8.647230329801756, 'chrf2': 39.55321473399512}

## 3. Modeling

In [ ]:
config = Config()

The device cuda is being used.


In [ ]:
config.model_checkpoint = 'Helsinki-NLP/opus-mt-tc-bible-big-deu_eng_fra_por_spa-itc'
config.source_lang = 'es'
config.target_lang = 'ast'

In [ ]:
model_config = MarianConfig.from_pretrained(config.model_checkpoint)
tokenizer = MarianTokenizer.from_pretrained(config.model_checkpoint)
model_checkpoint = MarianMTModel.from_pretrained(config.model_checkpoint, config=model_config)
model_checkpoint.to(config.device)

In [ ]:
config.model_max_length = tokenizer.model_max_length
config.max_source_length = tokenizer.model_max_length
config.max_target_length = tokenizer.model_max_length
config.label_pad_token_id = tokenizer.pad_token_id
config.task_prefix = ">>spa<< "
config.task_prefix_2 = ">>ast<< "
config.num_beams = 4
config.batch_size = 1

In [ ]:
print(f"Number of parameters: {model_checkpoint.num_parameters()}")

Number of parameters: 224993280


In [ ]:
# from peft import PeftModelForSeq2SeqLM, get_peft_config #maybe not use!

In [ ]:
# lora_config = {
#     "peft_type": "LORA",
#     "task_type": "SEQ_2_SEQ_LM",
#     "r": 64, # not much difference in training time... fine-tune full model?
#     "target_modules": ["q_proj", "v_proj", "q_proj"],
#     "lora_alpha": 32, #fix to 16
#     "fan_in_fan_out": False,
#     "bias": "none",
#     "lora_dropout": 0.1
# }

In [ ]:
# peft_config = get_peft_config(lora_config)
# peft_model = PeftModelForSeq2SeqLM(model_checkpoint, peft_config)
# peft_model.print_trainable_parameters()

In [ ]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds

  preds = np.where(
      preds != -100,
      preds,
      tokenizer.pad_token_id
  )

  predictions = tokenizer.batch_decode(
      preds,
      skip_special_tokens=True)

  labels = np.where(
      labels != -100,
      labels,
      tokenizer.pad_token_id)

  references = tokenizer.batch_decode(
      labels,
      skip_special_tokens=True)

  references = [[ref] for ref in references]

  results = sacrebleu_score.compute(predictions=predictions, references=references)
  results = {'sacrebleu': results['score']}

  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
  results['gen_len'] = np.mean(prediction_lens)

  chrf_sc = chrf.compute(predictions=predictions, references=references, word_order=2, lowercase=True)
  results['chrf2'] = chrf_sc['score']

  return results

In [ ]:
def preprocess_fn(batch): # task prefix
    sources = batch[config.source_lang]
    sources = [config.task_prefix + x for x in sources]
    targets = batch[config.target_lang]
    targets = [config.task_prefix_2 + x for x in targets]

    model_inputs = tokenizer(
        sources,
        max_length=config.max_source_length,
        truncation=True,
        # return_tensors='pt'
        )

    labels = tokenizer(
        targets,
        max_length=config.max_target_length,
        truncation=True)

    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [ ]:
dataset_dict = dataset_dict.map(
    preprocess_fn,
    batched=True,
    remove_columns=dataset_dict['train'].column_names)

Map:   0%|          | 0/122449 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model_checkpoint,
    label_pad_token_id=config.label_pad_token_id)

In [ ]:
config.save_steps = 500 # 4_500 4_000
config.lr = 5e-4
config.weight_decay = 0.001
config.num_epochs = 3

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir=fine_tuned_model_checkpoint,
    seed = config.seed,
    auto_find_batch_size = True,
    do_train = config.do_train,
    logging_steps = config.save_steps,
    eval_strategy ="steps",
    eval_steps = config.save_steps,
    save_steps = config.save_steps,
    save_total_limit = 3,
    report_to = "wandb",
    predict_with_generate = True,
    gradient_accumulation_steps=8, # reduce? 8 -> 4
    learning_rate = config.lr, # 5e-5, # default, config.lr, # toggle.. 5e-4 problematic?
    weight_decay = config.weight_decay,
    num_train_epochs = config.num_epochs,
    generation_num_beams = config.num_beams,
    fp16=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model_checkpoint,
    args=args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['dev'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #early stopping and load best model
)

In [ ]:
last_checkpoint = None

if os.path.isdir(fine_tuned_model_checkpoint):
  last_checkpoint = get_last_checkpoint(fine_tuned_model_checkpoint)

if last_checkpoint is not None:
  print(f"Checkpoint detected. Resuming training at {last_checkpoint}")

Checkpoint detected. Resuming training at ./gdrive/MyDrive/translation/opus-mt-tc-bible-big-deu_eng_fra_por_spa-itc_en-es/checkpoint/checkpoint-2000


In [ ]:
last_checkpoint=None

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
# rm = '/content/gdrive/MyDrive/translation/opus-mt-tc-bible-big-mul-mul_en-es'
# shutil.rmtree(rm)

In [ ]:
if trainer.args.do_train:
  start_time = perf_counter()
  train_result = trainer.train(resume_from_checkpoint=last_checkpoint)
  end_time = perf_counter()

  print(f"Elapsed training time {end_time - start_time}")
  trainer.save_model(fine_tuned_model_checkpoint)

Step,Training Loss,Validation Loss,Sacrebleu,Gen Len,Chrf2
500,0.337100,2.193467,27.254522,47.599799,48.658248
1000,0.325000,2.063804,27.763000,47.261785,49.416948
1500,0.303200,2.147596,28.024178,47.202608,49.631045
2000,0.274300,2.106616,27.926571,47.616851,49.874724
2500,0.248800,1.997771,28.204162,47.637914,50.143117
3000,0.234400,1.957083,28.355529,47.405216,50.285223
3500,0.219400,1.941396,28.409923,47.507523,50.343122
4000,0.199900,1.915902,28.643310,47.473420,50.502605
4500,0.167900,1.893328,28.539495,47.498495,50.550013
5000,0.162600,1.845672,28.685795,47.547643,50.589147


Elapsed training time 10341.264021293999


In [ ]:
metrics = train_result.metrics
metrics['train_samples'] = len(dataset_dict['train'])
trainer.save_metrics('train', metrics)
trainer.save_state()

In [ ]:
len(dataset_dict['dev'])

997

In [ ]:
metrics = trainer.evaluate(
    max_length=tokenizer.model_max_length,
    num_beams=config.num_beams,
    metric_key_prefix="eval_",
)

metrics['eval_samples'] = len(dataset_dict['dev'])

trainer.log_metrics('eval', metrics)
trainer.save_metrics('eval', metrics)
trainer.save_state()  # Ensures trainer state is saved SEE HERE AGAIN

***** eval metrics *****
  epoch                    =     2.9998
  eval__chrf2              =    50.6592
  eval__gen_len            =    47.4965
  eval__loss               =     1.8201
  eval__runtime            = 0:03:36.46
  eval__sacrebleu          =    28.6593
  eval__samples_per_second =      4.606
  eval__steps_per_second   =      0.577
  eval_samples             =        997


In [ ]:
trainer.state.log_history[-1]["eval_samples"] = len(dataset_dict['dev'])

In [ ]:
trainer.state.log_history[-1]

{'eval__loss': 1.8201063871383667,
 'eval__sacrebleu': 28.659270382358148,
 'eval__gen_len': 47.496489468405215,
 'eval__chrf2': 50.65921542296703,
 'eval__runtime': 216.4626,
 'eval__samples_per_second': 4.606,
 'eval__steps_per_second': 0.577,
 'epoch': 2.999804011236689,
 'step': 5739,
 'eval_samples': 997}

In [ ]:
devtest_ast = load_dataset('text', data_files='gdrive/MyDrive/translation/FLORES+/devtest/devtest.ast_Latn', split='train')
devtest_spa = load_dataset('text', data_files='gdrive/MyDrive/translation/FLORES+/devtest/devtest.spa_Latn', split='train')
# devtest = concatenate_datasets([devtest_spa, devtest_ast], axis=1)

In [ ]:
devtest_ast = devtest_ast.rename_column('text', 'ast')
devtest_spa = devtest_spa.rename_column('text', 'es')

In [ ]:
test_ds = concatenate_datasets([devtest_spa, devtest_ast], axis=1)

In [ ]:
device = torch.device('cuda')

In [ ]:
test_ds

Dataset({
    features: ['es', 'ast'],
    num_rows: 1012
})

In [ ]:
dataset_dict = dataset_dict.map(
    preprocess_fn,
    batched=True,
    remove_columns=dataset_dict['train'].column_names)

In [ ]:
test_ds = test_ds.map(
    preprocess_fn,
    batched=True,
    remove_columns=test_ds.column_names
)

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]

In [ ]:
metrics = trainer.evaluate(
    eval_dataset=test_ds,
    max_length=tokenizer.model_max_length,
    num_beams=config.num_beams,
    metric_key_prefix="test_",
)

In [ ]:
metrics['test_samples'] = len(test_ds)

trainer.log_metrics('test', metrics)
trainer.save_metrics('test', metrics)

***** test metrics *****
  epoch                    =     2.9998
  test__chrf2              =    50.6823
  test__gen_len            =    49.7826
  test__loss               =      1.842
  test__runtime            = 0:03:57.53
  test__sacrebleu          =    28.0745
  test__samples_per_second =       4.26
  test__steps_per_second   =      0.535
  test_samples             =       1012


In [ ]:
trainer.state.log_history

[{'loss': 0.3371,
  'grad_norm': 0.9933468103408813,
  'learning_rate': 0.00045643840390311905,
  'epoch': 0.26131835108120466,
  'step': 500},
 {'eval_loss': 2.193467378616333,
  'eval_sacrebleu': 27.25452244063354,
  'eval_gen_len': 47.599799398194584,
  'eval_chrf2': 48.65824783386484,
  'eval_runtime': 213.7513,
  'eval_samples_per_second': 4.664,
  'eval_steps_per_second': 0.585,
  'epoch': 0.26131835108120466,
  'step': 500},
 {'loss': 0.325,
  'grad_norm': 1.225108027458191,
  'learning_rate': 0.00041287680780623803,
  'epoch': 0.5226367021624093,
  'step': 1000},
 {'eval_loss': 2.0638043880462646,
  'eval_sacrebleu': 27.762999896365063,
  'eval_gen_len': 47.2617853560682,
  'eval_chrf2': 49.41694779322967,
  'eval_runtime': 216.2414,
  'eval_samples_per_second': 4.611,
  'eval_steps_per_second': 0.578,
  'epoch': 0.5226367021624093,
  'step': 1000},
 {'loss': 0.3032,
  'grad_norm': 1.1556296348571777,
  'learning_rate': 0.0003693152117093571,
  'epoch': 0.783955053243614,
  'st

## ?. Inference

In [ ]:
#0. save metrics
#1. trainer.evaluate()
# 2. generate_translation()
#save kwargs
#trainer.create_model_card()

In [ ]:
output_dir = '/content/gdrive/MyDrive/translation/opus-mt-tc-bible-big-deu_eng_fra_por_spa-itc_en-es/checkpoint/checkpoint-5739'
tokenizer = MarianTokenizer.from_pretrained(
output_dir,
local_files_only=True,
)
model = MarianMTModel.from_pretrained(
output_dir,
local_files_only=True,
)
model = model.to(device)
# maybe clean up (again) and then continue training
# test hypothesis on Galician / model